# Scraping data from the web (Part 1):

In [1]:
import pandas as pd
import numpy as np
import pyodbc
from bs4 import BeautifulSoup as bs
import requests
import json
import time

In [2]:
#read in data from site
plant_soup=bs(requests.get('https://www.thespruce.com/plants-a-to-z-5116344').text,'html5lib')

In [3]:
#find the JSON data generated from the javascript on the site
site_lst = plant_soup.find_all('script')

In [4]:
#select JSON data 
json_string = site_lst[3].string
json_string = json_string.split('}') 

In [5]:
cleaned_lst=[]
plant_site_lst=[]

#strip down to a single URL for each listed plant
for i,json_str in enumerate(json_string):
    cleaned_lst.append(json_str.replace('\n','').split(','))
    if len(cleaned_lst[i]) == 4:
        plant_site_lst.append(cleaned_lst[i][3])

#clean up each URL
plant_site_lst=[i.strip('"url" :') for i in plant_site_lst]

In [6]:
#create empty lists for plant data
botanical_name=[]
common_name=[]
family=[]
plant_type=[]
mature_size=[]
sun_exposure=[]
soil_type=[]
soil_ph=[]
bloom_time=[]
flower_color=[]
hardiness_zones=[]
native_area=[]

#read in data from site and filter it to correct list
for i in plant_site_lst: 
    time.sleep(.5) #wait n seconds to not overload the website with requests
    sample = bs(requests.get(i).text,'html5lib')
    plant_tbl_data = sample.find_all('td')
    
    for j,data in enumerate(plant_tbl_data):

        if plant_tbl_data[j].get_text().strip() == 'Botanical Name':
            botanical_name.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Common Name':
            common_name.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Family':
            family.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Plant Type':
            plant_type.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Mature Size':
            mature_size.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Sun Exposure':
            sun_exposure.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Soil Type':
            soil_type.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Soil pH':
            soil_ph.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Bloom Time':
            bloom_time.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Flower Color':
            flower_color.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Hardiness Zones':
            hardiness_zones.append(plant_tbl_data[j+1].get_text())
            
        elif plant_tbl_data[j].get_text().strip() == 'Native Area':
            native_area.append(plant_tbl_data[j+1].get_text())

    #check length of all lists and if not equal -> add nan to it
    #this should bring each list in line and an added nan should not exceed 1 per list
    lst_length = max(len(botanical_name),
                    len(common_name),
                    len(family),
                    len(plant_type),
                    len(mature_size),
                    len(sun_exposure),
                    len(soil_type),
                    len(soil_ph),
                    len(bloom_time),
                    len(flower_color),
                    len(hardiness_zones),
                    len(native_area))

    if len(botanical_name) < lst_length:
        botanical_name.append(np.nan)
    if len(common_name) < lst_length:
        common_name.append(np.nan)
    if len(family) < lst_length:
        family.append(np.nan)
    if len(plant_type) < lst_length:
        plant_type.append(np.nan)            
    if len(mature_size) < lst_length:
        mature_size.append(np.nan)
    if len(sun_exposure) < lst_length:
        sun_exposure.append(np.nan)
    if len(soil_type) < lst_length:
        soil_type.append(np.nan)
    if len(soil_ph) < lst_length:
        soil_ph.append(np.nan)
    if len(bloom_time) < lst_length:
        bloom_time.append(np.nan)           
    if len(flower_color) < lst_length:
        flower_color.append(np.nan)
    if len(hardiness_zones) < lst_length:
        hardiness_zones.append(np.nan)
    if len(native_area) < lst_length:
        native_area.append(np.nan)
            
#add all of the data to a data frame
plant_df = pd.DataFrame()
plant_df['botanical_name']=botanical_name
plant_df['common_name']=common_name
plant_df['family']=family
plant_df['plant_type']=plant_type
plant_df['mature_size']=mature_size
plant_df['sun_exposure']=sun_exposure
plant_df['soil_type']=soil_type
plant_df['soil_ph']=soil_ph
plant_df['bloom_time']=bloom_time
plant_df['flower_color']=flower_color
plant_df['hardiness_zones']=hardiness_zones
plant_df['native_area']=native_area


In [11]:
#since the key for the database will use the latin name, any missing their latin name will be dropped. 
plant_df.dropna(subset=['botanical_name'],inplace=True)
plant_df.reset_index(drop=True, inplace=True)

#save the scrapped date to a csv
plant_df.to_csv('Scrapped Plant Data.csv')